In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

In [2]:
CUE_ON_T = 1.0
SAMPLE_ON_T = SACCADE_ON_TIME = 3.0
MNM_END_TIME = 3.0
SUBREGIONS = {'Mid-Dorsal': 'MD', 'Posterior-Ventral': 'PV',
              'Anterior-Dorsal': 'AD', 'Posterior-Dorsal': 'PD', 'Anterior-Ventral': 'AV'}

def mat_to_df(dg, monkey, area, num_stimuli=1, numeric_cell=False, ):
    rows = []
    cells = []
    for c in range(dg.shape[0]):
        c_num = str(c + 1).zfill(3)
        cell_name = f'{monkey}-{area}-{c_num}'
        # cells.append([c + 1 if numeric_cell else cell_name, monkey, area])
        for p in range(dg.shape[1]):
            feature_names = dg[c][p][0].dtype.names
            for t in range(dg[c, p].shape[1]):
                cue_on_t = dg[c, p][0, t]['Cue_onT'][0][0]
                ts = dg[c, p][0, t]['TS'].flatten()
                if num_stimuli == 1:
                    ts = ts - cue_on_t + CUE_ON_T
                    ts = np.array([t for t in ts if t >= 0 and t <=
                                  SACCADE_ON_TIME + 1.0], dtype=np.float32)
                    if (len(ts) > 0):
                        rows.append([cell_name, p, t, ts])  # , cue_on_t])
                else:
                    sample_on_t = dg[c, p][0, t]['Sample_onT'][0][0]
                    ts2 = np.array([SAMPLE_ON_T + t - sample_on_t for t in ts if t >
                                   sample_on_t and t <= sample_on_t + MNM_END_TIME], dtype=np.float32)
                    ts = np.array([t - cue_on_t + CUE_ON_T for t in ts if t >= cue_on_t -
                                  CUE_ON_T and t <= cue_on_t + SAMPLE_ON_T - CUE_ON_T], dtype=np.float32)
                    ts = np.concatenate((ts, ts2))
                    try:
                        ismatch = dg[c, p][0, t]['IsMatch'][0][0]
                        if len(ts) > 0:
                            rows.append([c + 1, p, t, ts, ismatch])
                    except:
                        pass
 
    df = pd.DataFrame(rows, columns=[
                      'cell', 'position', 'trial', 'ts'] + ([] if num_stimuli == 1 else ['ismatch']))
    # df = df.set_index(['cell', 'position', 'trial'])
    # , pd.DataFrame(cells, columns=['cell', 'monkey', 'area']).set_index('cell')
    return df

In [3]:
def get_cells(df, style='monkey-area'):
    cells = df.cell.unique()
    cells = pd.DataFrame(cells, columns=['cell'])
    cells['monkey'] = cells.cell.apply(lambda x: x.split('-')[0])
    cells['area'] = cells.cell.apply(lambda x: x.split('-')[1])
    if 'subregion' in style:
        cells['subregion'] = cells.cell.apply(lambda x: x.split('-')[2])
    if 'phase' in style:
        cells['phase'] = cells.cell.apply(lambda x: x.split('-')[3])
    cells = cells.set_index('cell')
    return cells


In [4]:
def load_mnm_spatial():
    asd_mat = loadmat(
        '../../../data/MNM_original_data/all_spatial_data.mat')['all_spatial_data']
    asi_mat = loadmat(
        '../../../data/MNM_original_data/all_spatial_info.mat')['all_spatial_info']

    asd_mat = asd_mat[:, :8]
    asi_mat = asi_mat[:, [0, 3, 4]]
    extract = np.vectorize(lambda x: x[0])
    asi_mat = extract(asi_mat)

    asi_df = pd.DataFrame(asi_mat, columns=['monkey', 'phase', 'subregion'])
    asi_df.monkey = asi_df.monkey.apply(lambda x: x[0:3].upper())
    asi_df.subregion = asi_df.subregion.apply(lambda x: SUBREGIONS[x].upper())

    asd_df = mat_to_df(asd_mat, '<REPLACE>', 'PFC',
                        num_stimuli=2, numeric_cell=True)

    df = pd.merge(asd_df, asi_df, left_on='cell', right_index=True)
    df['cell'] = df.apply(lambda r: '-'.join([r.monkey, 'PFC',
                          str(r.subregion), r.phase, str(r.cell)]), axis=1)
    cells = get_cells(df, style='monkey-area-subregion-phase')
    df = df.drop(columns=['monkey', 'subregion', 'phase'])

    return {'raw_df': df.set_index(['cell', 'position', 'trial']), 'cells': cells}

In [5]:
df_dict = load_mnm_spatial()

In [6]:
raw_df = pd.DataFrame(df_dict['raw_df'])

In [7]:
cells = pd.DataFrame(df_dict['cells'])

In [8]:
# Resetting the index to move 'cell', 'position', and 'trial' back to columns
raw_df = raw_df.reset_index()

# Verify the change by printing the columns
# raw_df.sample(10)

In [9]:
df = pd.merge(raw_df, cells, on='cell', how='inner')
df.sample(3)

,cell,position,trial,ts,ismatch,monkey,area,subregion,phase
135700,ADR-PFC-PD-PRE-1077,0,2,"[0.79045, 5.19365]",0,ADR,PFC,PD,PRE
314911,NIN-PFC-AD-POST-2895,1,12,"[2.727425, 3.03115, 3.099975, 3.17825, 4.25105]",0,NIN,PFC,AD,POST
214494,ADR-PFC-PV-POST-1891,0,11,"[2.4869, 2.742575, 5.82365]",1,ADR,PFC,PV,POST


In [10]:
lengths = df.ts.apply(lambda x: len(x))
# lengths.value_counts()

In [11]:
filtered_df = df
filtered_df['ts_length'] = filtered_df['ts'].apply(len)
average_ts_length = filtered_df.groupby('cell')['ts_length'].mean().reset_index()
sum_ts_length = filtered_df.groupby('cell')['ts_length'].sum().reset_index()
use_cell = average_ts_length[(average_ts_length.ts_length >= 100) & (sum_ts_length.ts_length >= 500)].cell

In [12]:
filtered_df = filtered_df[filtered_df['cell'].isin(use_cell)]
filtered_df.shape

(12980, 10)

In [13]:
pre_df = filtered_df[filtered_df.phase == "PRE"]
post_df = filtered_df[filtered_df.phase == "POST"]
pre_df.shape, post_df.shape

((6497, 10), (6483, 10))

In [22]:
import torch

def get_soft_one_hot(pos):
    return torch.tensor(
        [
            0.6 if pos == x else (0.2 if abs(x - pos) ==
                                  1 or abs(x - pos) == 7 else 0)
            for x in range(1, 9)
        ]
    )


def create_y(l, func):
    return torch.hstack(
        [
            torch.stack([func(y) for y in l]),
            torch.tensor(l).unsqueeze(1),
        ]
    )

def window_data(df, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS):
    df = df.sort_values(['cell', 'ismatch']).reset_index(drop=True)

    window_data = []
    for start in np.arange(1, 3.5, WINDOW_STRIDE):
        window_data.append(df.ts.apply(lambda x: (
            (x >= start) & (x < start + WINDOW_WIDTH)).sum()).array)

    window_d_t = torch.tensor(window_data).T
    BINS = window_d_t.shape[1]

    X = torch.zeros((NUM_OF_SAMPLES * NUM_POSITIONS,
                    NUM_OF_CELLS * BINS), dtype=torch.float32)
    y = []
    for p in range(1, NUM_POSITIONS+1):
        y += [p] * NUM_OF_SAMPLES
        cells_idxs = df[df.position == p-1].groupby("cell", sort=False).apply(
            lambda x: x.index)
        for i, cell_idxs in enumerate(cells_idxs):
            idxs = np.random.choice(cell_idxs, NUM_OF_SAMPLES)
            for j, idx in enumerate(idxs):
                X[(p - 1) * NUM_OF_SAMPLES + j, i *
                  BINS: (i + 1) * BINS] = window_d_t[idx]

    return X, create_y(y, get_soft_one_hot)

In [23]:
from sklearn.model_selection import train_test_split 

NUM_OF_CELLS = pre_df.cell.nunique()
NUM_OF_SAMPLES = 40
WINDOW_STRIDE = 0.4
WINDOW_WIDTH = 0.4
NUM_POSITIONS = 2

X, y = window_data(pre_df, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) 

In [24]:
X.shape

torch.Size([80, 378])

In [25]:
X_train.shape, X_test.shape

(torch.Size([64, 378]), torch.Size([16, 378]))

In [26]:
y.shape

torch.Size([80, 9])

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleFNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X.shape[1]
num_classes = y_train_labels.max().item() + 1

model = SimpleFNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/20], Loss: 2.7215, Accuracy: 51.56%
Epoch [2/20], Loss: 1.3895, Accuracy: 59.38%
Epoch [3/20], Loss: 0.7394, Accuracy: 78.12%
Epoch [4/20], Loss: 0.5624, Accuracy: 81.25%
Epoch [5/20], Loss: 0.5582, Accuracy: 79.69%
Epoch [6/20], Loss: 0.1844, Accuracy: 90.62%
Epoch [7/20], Loss: 0.2455, Accuracy: 89.06%
Epoch [8/20], Loss: 0.1695, Accuracy: 92.19%
Epoch [9/20], Loss: 0.0327, Accuracy: 100.00%
Epoch [10/20], Loss: 0.0851, Accuracy: 95.31%
Epoch [11/20], Loss: 0.0765, Accuracy: 98.44%
Epoch [12/20], Loss: 0.0236, Accuracy: 100.00%
Epoch [13/20], Loss: 0.0157, Accuracy: 100.00%
Epoch [14/20], Loss: 0.0301, Accuracy: 100.00%
Epoch [15/20], Loss: 0.0276, Accuracy: 100.00%
Epoch [16/20], Loss: 0.0126, Accuracy: 100.00%
Epoch [17/20], Loss: 0.0064, Accuracy: 100.00%
Epoch [18/20], Loss: 0.0047, Accuracy: 100.00%
Epoch [19/20], Loss: 0.0056, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0073, Accuracy: 100.00%
Training Finished.


In [26]:
# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0123, Test Accuracy: 100.00%
Final Test Loss: 0.0123, Final Test Accuracy: 100.00%


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleCNN(nn.Module):
    def __init__(self, num_cells, bins, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_cells, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

BINS = 7
num_cells = X.shape[1] // BINS
bins = BINS
num_classes = y_train_labels.max().item() + 1

model = SimpleCNN(num_cells, bins, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), num_cells, bins)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')

Epoch [1/30], Loss: 1.0990, Accuracy: 51.67%
Epoch [2/30], Loss: 0.8720, Accuracy: 51.67%
Epoch [3/30], Loss: 0.7603, Accuracy: 61.67%
Epoch [4/30], Loss: 0.6785, Accuracy: 70.00%
Epoch [5/30], Loss: 0.6282, Accuracy: 78.33%
Epoch [6/30], Loss: 0.6001, Accuracy: 76.67%
Epoch [7/30], Loss: 0.5441, Accuracy: 76.67%
Epoch [8/30], Loss: 0.4990, Accuracy: 91.67%
Epoch [9/30], Loss: 0.4376, Accuracy: 93.33%
Epoch [10/30], Loss: 0.3727, Accuracy: 93.33%
Epoch [11/30], Loss: 0.3107, Accuracy: 96.67%
Epoch [12/30], Loss: 0.2439, Accuracy: 98.33%
Epoch [13/30], Loss: 0.1886, Accuracy: 100.00%
Epoch [14/30], Loss: 0.1446, Accuracy: 100.00%
Epoch [15/30], Loss: 0.1053, Accuracy: 100.00%
Epoch [16/30], Loss: 0.0734, Accuracy: 100.00%
Epoch [17/30], Loss: 0.0514, Accuracy: 100.00%
Epoch [18/30], Loss: 0.0358, Accuracy: 100.00%
Epoch [19/30], Loss: 0.0242, Accuracy: 100.00%
Epoch [20/30], Loss: 0.0161, Accuracy: 100.00%
Epoch [21/30], Loss: 0.0116, Accuracy: 100.00%
Epoch [22/30], Loss: 0.0086, Accur

In [28]:
# Evaluation function
def evaluate_model(model, test_loader, criterion, num_cells, bins):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), num_cells, bins)  # Reshape the inputs to match the CNN input size
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, num_cells, bins)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0013, Test Accuracy: 100.00%
Final Test Loss: 0.0013, Final Test Accuracy: 100.00%


In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, num_features, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.relu(x[:, -1, :])
        x = self.fc(x)
        return x

num_features = BINS
sequence_length = len(pre_df.cell.unique())
hidden_size = 64
num_layers = 2
num_classes = y_train_labels.max().item() + 1

model = LSTMModel(num_features, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), sequence_length, num_features)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/100], Loss: 1.0499, Accuracy: 46.67%
Epoch [2/100], Loss: 1.0141, Accuracy: 66.67%
Epoch [3/100], Loss: 0.9781, Accuracy: 56.67%
Epoch [4/100], Loss: 0.9396, Accuracy: 53.33%
Epoch [5/100], Loss: 0.9014, Accuracy: 53.33%
Epoch [6/100], Loss: 0.8654, Accuracy: 53.33%
Epoch [7/100], Loss: 0.8267, Accuracy: 53.33%
Epoch [8/100], Loss: 0.8000, Accuracy: 53.33%
Epoch [9/100], Loss: 0.7755, Accuracy: 53.33%
Epoch [10/100], Loss: 0.7581, Accuracy: 53.33%
Epoch [11/100], Loss: 0.7435, Accuracy: 53.33%
Epoch [12/100], Loss: 0.7334, Accuracy: 53.33%
Epoch [13/100], Loss: 0.7246, Accuracy: 53.33%
Epoch [14/100], Loss: 0.7166, Accuracy: 53.33%
Epoch [15/100], Loss: 0.7091, Accuracy: 56.67%
Epoch [16/100], Loss: 0.7029, Accuracy: 56.67%
Epoch [17/100], Loss: 0.6973, Accuracy: 56.67%
Epoch [18/100], Loss: 0.6907, Accuracy: 56.67%
Epoch [19/100], Loss: 0.6891, Accuracy: 55.00%
Epoch [20/100], Loss: 0.6875, Accuracy: 55.00%
Epoch [21/100], Loss: 0.6733, Accuracy: 66.67%
Epoch [22/100], Loss: 

Epoch [23/100], Loss: 0.6573, Accuracy: 71.67%
Epoch [24/100], Loss: 0.6511, Accuracy: 63.33%
Epoch [25/100], Loss: 0.6403, Accuracy: 70.00%
Epoch [26/100], Loss: 0.6288, Accuracy: 81.67%
Epoch [27/100], Loss: 0.6143, Accuracy: 78.33%
Epoch [28/100], Loss: 0.5986, Accuracy: 78.33%
Epoch [29/100], Loss: 0.5877, Accuracy: 83.33%
Epoch [30/100], Loss: 0.5898, Accuracy: 73.33%
Epoch [31/100], Loss: 0.5465, Accuracy: 83.33%
Epoch [32/100], Loss: 0.5506, Accuracy: 88.33%
Epoch [33/100], Loss: 0.5591, Accuracy: 71.67%
Epoch [34/100], Loss: 0.5127, Accuracy: 88.33%
Epoch [35/100], Loss: 0.5066, Accuracy: 91.67%
Epoch [36/100], Loss: 0.4760, Accuracy: 88.33%
Epoch [37/100], Loss: 0.4677, Accuracy: 88.33%
Epoch [38/100], Loss: 0.4298, Accuracy: 93.33%
Epoch [39/100], Loss: 0.4271, Accuracy: 91.67%
Epoch [40/100], Loss: 0.3949, Accuracy: 95.00%
Epoch [41/100], Loss: 0.3620, Accuracy: 96.67%
Epoch [42/100], Loss: 0.3483, Accuracy: 96.67%
Epoch [43/100], Loss: 0.3216, Accuracy: 96.67%
Epoch [44/100

In [30]:
# Evaluation function for LSTM model
def evaluate_model(model, test_loader, criterion, sequence_length, num_features):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), sequence_length, num_features)  # Reshape inputs
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, sequence_length, num_features)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0152, Test Accuracy: 100.00%
Final Test Loss: 0.0152, Final Test Accuracy: 100.00%


In [28]:
from sklearn.model_selection import train_test_split 

NUM_OF_CELLS = post_df.cell.nunique()
NUM_OF_SAMPLES = 40
WINDOW_STRIDE = 0.4
WINDOW_WIDTH = 0.4
NUM_POSITIONS = 2

X, y = window_data(post_df, NUM_OF_SAMPLES, WINDOW_STRIDE, WINDOW_WIDTH, NUM_POSITIONS, NUM_OF_CELLS)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True) 

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleFNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

input_size = X.shape[1]
num_classes = y_train_labels.max().item() + 1

model = SimpleFNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/20], Loss: 1.6060, Accuracy: 31.25%
Epoch [2/20], Loss: 0.6778, Accuracy: 71.88%
Epoch [3/20], Loss: 0.4185, Accuracy: 82.81%
Epoch [4/20], Loss: 0.2822, Accuracy: 84.38%
Epoch [5/20], Loss: 0.1491, Accuracy: 93.75%
Epoch [6/20], Loss: 0.2013, Accuracy: 89.06%
Epoch [7/20], Loss: 0.0516, Accuracy: 100.00%
Epoch [8/20], Loss: 0.1778, Accuracy: 92.19%
Epoch [9/20], Loss: 0.0327, Accuracy: 100.00%
Epoch [10/20], Loss: 0.0770, Accuracy: 98.44%
Epoch [11/20], Loss: 0.0677, Accuracy: 96.88%
Epoch [12/20], Loss: 0.0080, Accuracy: 100.00%
Epoch [13/20], Loss: 0.0159, Accuracy: 100.00%
Epoch [14/20], Loss: 0.0342, Accuracy: 100.00%
Epoch [15/20], Loss: 0.0149, Accuracy: 100.00%
Epoch [16/20], Loss: 0.0041, Accuracy: 100.00%
Epoch [17/20], Loss: 0.0028, Accuracy: 100.00%
Epoch [18/20], Loss: 0.0051, Accuracy: 100.00%
Epoch [19/20], Loss: 0.0082, Accuracy: 100.00%
Epoch [20/20], Loss: 0.0076, Accuracy: 100.00%
Training Finished.


In [30]:
# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0053, Test Accuracy: 100.00%
Final Test Loss: 0.0053, Final Test Accuracy: 100.00%


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class SimpleCNN(nn.Module):
    def __init__(self, num_cells, bins, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_cells, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

BINS = 7
num_cells = X.shape[1] // BINS
bins = BINS
num_classes = y_train_labels.max().item() + 1

model = SimpleCNN(num_cells, bins, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), num_cells, bins)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')

Epoch [1/30], Loss: 0.9554, Accuracy: 51.56%
Epoch [2/30], Loss: 0.7760, Accuracy: 51.56%
Epoch [3/30], Loss: 0.6703, Accuracy: 57.81%
Epoch [4/30], Loss: 0.6314, Accuracy: 75.00%
Epoch [5/30], Loss: 0.5592, Accuracy: 81.25%
Epoch [6/30], Loss: 0.5155, Accuracy: 73.44%
Epoch [7/30], Loss: 0.4669, Accuracy: 90.62%
Epoch [8/30], Loss: 0.4187, Accuracy: 90.62%
Epoch [9/30], Loss: 0.3577, Accuracy: 90.62%
Epoch [10/30], Loss: 0.3018, Accuracy: 95.31%
Epoch [11/30], Loss: 0.2609, Accuracy: 96.88%
Epoch [12/30], Loss: 0.2141, Accuracy: 96.88%
Epoch [13/30], Loss: 0.1777, Accuracy: 98.44%
Epoch [14/30], Loss: 0.1381, Accuracy: 98.44%
Epoch [15/30], Loss: 0.1111, Accuracy: 98.44%
Epoch [16/30], Loss: 0.0884, Accuracy: 100.00%
Epoch [17/30], Loss: 0.0644, Accuracy: 100.00%
Epoch [18/30], Loss: 0.0511, Accuracy: 100.00%
Epoch [19/30], Loss: 0.0369, Accuracy: 100.00%
Epoch [20/30], Loss: 0.0278, Accuracy: 100.00%
Epoch [21/30], Loss: 0.0226, Accuracy: 100.00%
Epoch [22/30], Loss: 0.0142, Accuracy

In [32]:
# Evaluation function
def evaluate_model(model, test_loader, criterion, num_cells, bins):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), num_cells, bins)  # Reshape the inputs to match the CNN input size
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, num_cells, bins)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0027, Test Accuracy: 100.00%
Final Test Loss: 0.0027, Final Test Accuracy: 100.00%


In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

y_train_labels = y_train[:, -1].long()

train_dataset = TensorDataset(X_train, y_train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

y_test_labels = y_test[:, -1].long()

test_dataset = TensorDataset(X_test, y_test_labels)
test_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

class LSTMModel(nn.Module):
    def __init__(self, num_features, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=num_features, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.relu(x[:, -1, :])
        x = self.fc(x)
        return x

num_features = BINS
sequence_length = len(post_df.cell.unique())
hidden_size = 64
num_layers = 2
num_classes = y_train_labels.max().item() + 1

model = LSTMModel(num_features, hidden_size, num_layers, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.size(0), sequence_length, num_features)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

print('Training Finished.')


Epoch [1/100], Loss: 1.1021, Accuracy: 26.56%
Epoch [2/100], Loss: 1.0546, Accuracy: 46.88%
Epoch [3/100], Loss: 1.0074, Accuracy: 46.88%
Epoch [4/100], Loss: 0.9634, Accuracy: 46.88%
Epoch [5/100], Loss: 0.9235, Accuracy: 46.88%
Epoch [6/100], Loss: 0.8834, Accuracy: 46.88%
Epoch [7/100], Loss: 0.8445, Accuracy: 46.88%
Epoch [8/100], Loss: 0.8129, Accuracy: 46.88%
Epoch [9/100], Loss: 0.7825, Accuracy: 46.88%
Epoch [10/100], Loss: 0.7627, Accuracy: 54.69%
Epoch [11/100], Loss: 0.7474, Accuracy: 54.69%
Epoch [12/100], Loss: 0.7356, Accuracy: 53.12%
Epoch [13/100], Loss: 0.7273, Accuracy: 53.12%
Epoch [14/100], Loss: 0.7209, Accuracy: 53.12%
Epoch [15/100], Loss: 0.7155, Accuracy: 53.12%
Epoch [16/100], Loss: 0.7105, Accuracy: 53.12%
Epoch [17/100], Loss: 0.7059, Accuracy: 53.12%
Epoch [18/100], Loss: 0.7024, Accuracy: 53.12%
Epoch [19/100], Loss: 0.7005, Accuracy: 54.69%
Epoch [20/100], Loss: 0.6953, Accuracy: 62.50%


Epoch [21/100], Loss: 0.6914, Accuracy: 64.06%
Epoch [22/100], Loss: 0.6866, Accuracy: 57.81%
Epoch [23/100], Loss: 0.6862, Accuracy: 54.69%
Epoch [24/100], Loss: 0.6787, Accuracy: 56.25%
Epoch [25/100], Loss: 0.6706, Accuracy: 76.56%
Epoch [26/100], Loss: 0.6640, Accuracy: 71.88%
Epoch [27/100], Loss: 0.6567, Accuracy: 70.31%
Epoch [28/100], Loss: 0.6475, Accuracy: 76.56%
Epoch [29/100], Loss: 0.6396, Accuracy: 78.12%
Epoch [30/100], Loss: 0.6302, Accuracy: 78.12%
Epoch [31/100], Loss: 0.6190, Accuracy: 73.44%
Epoch [32/100], Loss: 0.6045, Accuracy: 76.56%
Epoch [33/100], Loss: 0.5928, Accuracy: 81.25%
Epoch [34/100], Loss: 0.5787, Accuracy: 81.25%
Epoch [35/100], Loss: 0.5601, Accuracy: 79.69%
Epoch [36/100], Loss: 0.5495, Accuracy: 79.69%
Epoch [37/100], Loss: 0.5263, Accuracy: 81.25%
Epoch [38/100], Loss: 0.5048, Accuracy: 81.25%
Epoch [39/100], Loss: 0.4824, Accuracy: 85.94%
Epoch [40/100], Loss: 0.4837, Accuracy: 81.25%
Epoch [41/100], Loss: 0.4384, Accuracy: 85.94%
Epoch [42/100

In [35]:
# Evaluation function for LSTM model
def evaluate_model(model, test_loader, criterion, sequence_length, num_features):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), sequence_length, num_features)  # Reshape inputs
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_test_loss = test_loss / len(test_loader)
    accuracy = 100 * correct / total
    
    print(f'Test Loss: {avg_test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    return avg_test_loss, accuracy

# Evaluate the model after training
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, sequence_length, num_features)

print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.0133, Test Accuracy: 100.00%
Final Test Loss: 0.0133, Final Test Accuracy: 100.00%
